<a href="https://colab.research.google.com/github/ZachryRamkissoon/COMP-3610-Project/blob/main/COMP_3610_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [3]:
import pandas as pd
import re
import unicodedata

#Data Collection
Load the first CSV file containing years 2000-2024

In [4]:
try:
    df_spotify = pd.read_csv('billboard_24years_lyrics_spotify.csv', encoding='utf-8')
except UnicodeDecodeError:
    print("UTF-8 failed for billboard_24years_lyrics_spotify.csv, trying latin1...")
    df_spotify = pd.read_csv('billboard_24years_lyrics_spotify.csv', encoding='latin1')

Load the second CSV file containing years 1964-2015

In [5]:
try:
    df_1964_2015 = pd.read_csv('billboard_lyrics_1964-2015.csv', encoding='utf-8')
except UnicodeDecodeError:
    print("UTF-8 failed for billboard_lyrics_1964-2015.csv, trying latin1...")
    df_1964_2015 = pd.read_csv('billboard_lyrics_1964-2015.csv', encoding='latin1')

UTF-8 failed for billboard_lyrics_1964-2015.csv, trying latin1...


#Data Preprocessing

Filter for years 2016 to 2024

In [6]:
df_spotify_filtered = df_spotify[df_spotify['year'].between(2016, 2024)].copy()

Remove duplicate records based on all columns

In [7]:
df_spotify_filtered = df_spotify_filtered.drop_duplicates()

Function to clean lyrics

In [8]:
def clean_lyrics_ascii(text):
    if pd.isna(text):  # Handle missing values
        return ''
    text = text.lower() # Convert to lowercase

    text = re.sub(r'[^\x00-\x7F]+', ' ', text) # Remove non-ASCII characters

    text = re.sub(r'\s*\'\s*', '', text) # Remove apostrophes and merge contractions

    text = re.sub(r'[^a-z0-9\s]', ' ', text) # Remove all punctuation except spaces, keep letters and numbers

    text = re.sub(r'\d+embed$', '', text) # Remove number followed by "embed" at the end

    text = re.sub(r'\s+', ' ', text).strip() # Replace multiple spaces with a single space and remove leading/trailing whitespace

    return text

Remove non-Latin characters

In [9]:
df_spotify_filtered['lyrics_cleaned'] = df_spotify_filtered['lyrics'].apply(clean_lyrics_ascii)

Select and rename columns for consistency

In [10]:
df_spotify_cleaned = df_spotify_filtered[['song', 'year', 'ranking', 'lyrics_cleaned']]

Rename columns to match the first dataset

In [11]:
df_1964_2015 = df_1964_2015.rename(columns={
    'Song': 'song',
    'Year': 'year',
    'Rank': 'ranking',
    'Lyrics': 'lyrics_cleaned'
})

Select only the required columns

In [12]:
df_1964_2015_cleaned = df_1964_2015[['song', 'year', 'ranking', 'lyrics_cleaned']]

Combine the datasets

In [13]:
df_combined = pd.concat([df_1964_2015_cleaned, df_spotify_cleaned], ignore_index=True)

Remove duplicates across the combined dataset (based on all columns)

In [14]:
df_combined = df_combined.drop_duplicates()

Inspect the first few rows of the combined DataFrame

In [15]:
print("Combined Cleaned DataFrame (First 5 rows):")
print(df_combined.head())
print("\nOriginal vs Cleaned Lyrics (First 5 rows from 2016-2024 data):")
for i in range(min(5, len(df_spotify_filtered))):
    print(f"Song: {df_spotify_filtered['song'].iloc[i]} (Year: {df_spotify_filtered['year'].iloc[i]}, Rank: {df_spotify_filtered['ranking'].iloc[i]})")
    print(f"Original: {df_spotify_filtered['lyrics'].iloc[i]}")
    print(f"Cleaned: {df_spotify_filtered['lyrics_cleaned'].iloc[i]}")
    print("-" * 50)

Combined Cleaned DataFrame (First 5 rows):
                                       song  year  ranking  \
0                               wooly bully  1965        1   
1  i cant help myself sugar pie honey bunch  1965        2   
2                i cant get no satisfaction  1965        3   
3                       you were on my mind  1965        4   
4              youve lost that lovin feelin  1965        5   

                                      lyrics_cleaned  
0  sam the sham miscellaneous wooly bully wooly b...  
1   sugar pie honey bunch you know that i love yo...  
2                                                     
3   when i woke up this morning you were on my mi...  
4   you never close your eyes anymore when i kiss...  

Original vs Cleaned Lyrics (First 5 rows from 2016-2024 data):
Song: Love Yourself (Year: 2016, Rank: 1)
Original: 
For all the times that you rained on my parade
And all the clubs you get in using my name
You think you broke my heart, oh girl, for good

Save the combined cleaned data to a new CSV file

In [16]:
df_combined.to_csv('billboard_1964_2024_lyrics_cleaned.csv', index=False)
print("Combined cleaned data saved to 'billboard_1964_2024_lyrics_cleaned.csv'")
print(f"Number of unique records: {len(df_combined)}")

Combined cleaned data saved to 'billboard_1964_2024_lyrics_cleaned.csv'
Number of unique records: 5962
